In [2]:
import torch
import torch.nn.functional as F

from mmengine import Config
from mmpretrain.models import build_classifier

from projects.clip.datasets.tokenizer import tokenize
from projects.clip.datasets.metadata import OPENAI_IMAGENET_TEMPLATES, SIMPLE_IMAGENET_TEMPLATES

In [3]:
cfg = Config.fromfile('../projects/clip/config/vit-base-p16_pt-64xb64_in1k.py').model.backbone
cfg.init_cfg.checkpoint = '../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth'
model = build_classifier(cfg)
model.init_weights()
model.eval()

05/29 23:37:07 - mmengine - INFO - load model from: ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth
05/29 23:37:07 - mmengine - INFO - Loads checkpoint by local backend from path: ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth
05/29 23:37:10 - mmengine - INFO - 
visual_projection - torch.Size([768, 512]): 
PretrainedInit: load from ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/29 23:37:10 - mmengine - INFO - 
positional_embedding - torch.Size([77, 512]): 
PretrainedInit: load from ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/29 23:37:10 - mmengine - INFO - 
text_projection - torch.Size([512, 512]): 
PretrainedInit: load from ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/29 23:37:10 - mmengine - INFO - 
logit_scale - torch.Size([]): 
PretrainedInit: load from ../data/pretrained/clip/CLIP-ViT-B-16-laion2B-s34B-b88K/pretrain.pth 
 
05/29 23:37:10 - mmengine - 

In [4]:
name_path = '../data/metadata/Objects365_names_fix.csv'
names = []
with open(name_path, 'r') as f:
    for line in f:
        tmp = line.strip().split(',')
        names.append(tmp[2])

In [5]:
templates = SIMPLE_IMAGENET_TEMPLATES
num_templates = len(templates)

num_names = len(names)

use_format = isinstance(templates[0], str)
text = [t.format(n) if use_format else t(n) 
        for n in names for t in templates]
token = tokenize(text).reshape(num_names, num_templates, -1)

with torch.no_grad():
    text_feats = model.encode_text(token)

text_feats = F.normalize(text_feats.mean(dim=1), dim=-1)

In [6]:
token.shape, text_feats.shape

(torch.Size([365, 1, 77]), torch.Size([365, 512]))

In [1]:
import torch
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-16', pretrained='laion2b_s34b_b88k')
tokenizer = open_clip.get_tokenizer('ViT-B-16')

token = tokenizer(text)

with torch.no_grad(), torch.cuda.amp.autocast():
    x = model.encode_text(token)

/home/ubuntu/anaconda3/envs/mmdet_3x/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'text' is not defined

In [15]:
import os
from pathlib import Path

base_dir = os.getcwd()
Path(base_dir).parent / 'data'

PosixPath('/home/ubuntu/mnt/hxm/ovd/mmpretrain/data')

In [11]:
base_dir = os.getcwd()
base_dir

'/home/ubuntu/mnt/hxm/ovd/mmpretrain/notebooks'

In [20]:
def text_split(text, n):
    for i in range(0, len(text), n):
        yield text[i: i + n]

In [25]:
text = torch.rand(5, 8, 3)

In [29]:
for x in text_split(text, 1):
    print(x.shape)

torch.Size([1, 8, 3])
torch.Size([1, 8, 3])
torch.Size([1, 8, 3])
torch.Size([1, 8, 3])
torch.Size([1, 8, 3])
